# Inference

In [ ]:
import transformers
import torch
import pandas as pd
import numpy as np
import os
from transformers import AutoModelForCausalLM, AutoTokenizer,AdamW
from tqdm import tqdm
pd.set_option('display.max_colwidth', 1000)

# CUDA 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
!pip install peft
from peft import PeftConfig, PeftModel
peft_config = PeftConfig.from_pretrained("학습시켰던 모델 경로")
base_model_path = peft_config.base_model_name_or_path
print(base_model_path)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Your checkpoint path
checkpoint_path = "/"

# Load model and tokenizer from checkpoint_path
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
transformers_model = AutoModelForCausalLM.from_pretrained(base_model_path)
peft_model = PeftModel.from_pretrained(transformers_model, checkpoint_path)
peft_model.to(device)

In [ ]:
# 200자 생성 inference
output = peft_model.generate(input_ids=input["input_ids"], max_length=200)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

In [ ]:
# test data load for inference
test_data = pd.read_csv('test.csv')

# test data 질문 끝에 있는 불필요한 따옴표 없애기
test_data['질문'] = test_data['질문'].str.strip('"')

# 문장 분리 및 새 칼럼 생성
def split_question(row):
    # 문장 구분자로 분리
    parts = pd.Series(row['질문'].split('. ')).str.strip()  # 마침표 기준 분리
    if len(parts) < 2:
        parts = pd.Series(row['질문'].split('? ')).str.strip()  # 물음표 기준 분리

    # 분리된 문장을 새로운 칼럼에 할당
    if len(parts) >= 2:
        return pd.Series({'문장1': parts[0], '문장2': parts[1]})
    else:
        return pd.Series({'문장1': parts[0], '문장2': None})

# 적용
new_columns = test_data.apply(split_question, axis=1)
df = pd.concat([test_data, new_columns], axis=1)

In [ ]:
df_question_1 = df[df['문장2'].isnull()] # 질문 1개
df_question_2 = df[df['문장2'].notnull()] # 질문 2개

# test.csv의 '질문'에 대한 '답변'을 저장할 리스트
question_1_pred = []

# '질문' 컬럼의 각 질문에 대해 답변 생성
for test_question in tqdm(df_question_1['문장1']):
    prompt = f"주어진 질문에 대한 답을 출력, 질문: {test_question},답: "
    # 입력 질문을 토큰화하고 모델 입력 형태로 변환
    input_seq = tokenizer(prompt,return_tensors='pt').to(device)

    # 답변 생성
    output = peft_model.generate(
        input_ids = input_seq["input_ids"],
        max_length=250
    )

    # 생성된 텍스트(답변) 저장
    for generated_sequence in output:
        full_text = tokenizer.decode(generated_sequence, skip_special_tokens=True)
        # 질문과 답변의 사이를 나타내는 eos_token (</s>)를 찾아, 이후부터 출력
        answer_start = full_text.find("답: ") + len("답: ")
        answer_only = full_text[answer_start:].strip()
        answer_only = answer_only.replace('\n', ' ')
        question_1_pred.append(answer_only)

df_question_1['Answer'] = question_1_pred

In [ ]:
# 질문을 2개로 쪼갠 경우
# test.csv의 '질문'에 대한 '답변'을 저장할 리스트
question_2_pred_1 = []

# '질문' 컬럼의 각 질문에 대해 답변 생성
for test_question in tqdm(df_question_2['문장1']):
    prompt = f"주어진 질문에 대한 답을 출력, 질문: {test_question},답: "
    # 입력 질문을 토큰화하고 모델 입력 형태로 변환
    input_seq = tokenizer(prompt,return_tensors='pt').to(device)

    # 답변 생성
    output = peft_model.generate(
        input_ids = input_seq["input_ids"],
        max_length=150
    )

    # 생성된 텍스트(답변) 저장
    for generated_sequence in output:
        full_text = tokenizer.decode(generated_sequence, skip_special_tokens=True)
        # 질문과 답변의 사이를 나타내는 eos_token (</s>)를 찾아, 이후부터 출력
        answer_start = full_text.find("답: ") + len("답: ")
        answer_only = full_text[answer_start:].strip()
        answer_only = answer_only.replace('\n', ' ')
        question_2_pred_1.append(answer_only)

In [ ]:
# test.csv의 '질문'에 대한 '답변'을 저장할 리스트
question_2_pred_2 = []

# '질문' 컬럼의 각 질문에 대해 답변 생성
for test_question in tqdm(df_question_2['문장2']):
    prompt = f"주어진 질문에 대한 답을 출력, 질문: {test_question},답: "
    # 입력 질문을 토큰화하고 모델 입력 형태로 변환
    input_seq = tokenizer(prompt,return_tensors='pt').to(device)

    # 답변 생성
    output = peft_model.generate(
        input_ids = input_seq["input_ids"],
        max_length=150
    )

    # 생성된 텍스트(답변) 저장
    for generated_sequence in output:
        full_text = tokenizer.decode(generated_sequence, skip_special_tokens=True)
        # 질문과 답변의 사이를 나타내는 eos_token (</s>)를 찾아, 이후부터 출력
        answer_start = full_text.find("답: ") + len("답: ")
        answer_only = full_text[answer_start:].strip()
        answer_only = answer_only.replace('\n', ' ')
        question_2_pred_2.append(answer_only)

df_question_2['Answer1'] = question_2_pred_1
df_question_2['Answer2'] = question_2_pred_2
df_question_2["Answer"] = df_question_2["Answer1"] + df_question_2["Answer2"]
df_question_1 = df_question_1[['id', 'Answer']]
df_question_2 = df_question_2[['id', 'Answer']]

In [ ]:
# test_data와 df_question_1 병합
merged_1 = pd.merge(test_data, df_question_1, on='id', how='left', suffixes=('', '_q1'))
# test_data와 df_question_2 병합
merged_2 = pd.merge(test_data, df_question_2, on='id', how='left', suffixes=('', '_q2'))

# "Answer" 칼럼 채우기
# df_question_1의 "Answer" 값이 있는 경우 사용, 그렇지 않으면 df_question_2의 값을 사용
test_data['Answer'] = merged_1['Answer'].combine_first(merged_2['Answer'])

In [ ]:
# test data embedding
!pip install sentence_transformers

# Test 데이터셋의 모든 질의에 대한 답변으로부터 512 차원의 Embedding Vector 추출
# 평가를 위한 Embedding Vector 추출에 활용하는 모델은 'distiluse-base-multilingual-cased-v1' 이므로 반드시 확인해주세요.
from sentence_transformers import SentenceTransformer # SentenceTransformer Version 2.2.2

# Embedding Vector 추출에 활용할 모델(distiluse-base-multilingual-cased-v1) 불러오기
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

# 생성한 모든 응답(답변)으로부터 Embedding Vector 추출
preds_3 = test_data["Answer"]
pred_embeddings = model.encode(preds_3)
pred_embeddings.shape

In [ ]:
# Inference
QUESTION = "도배지에 곰팡이가 생겼을 때 높은 습도가 원인인 경우, 습기 관리는 어떻게 해야 할까요?"
prompt = f"주어진 질문에 대한 답을 출력, 질문: {QUESTION},답: "
input = tokenizer(prompt, return_tensors='pt').to(device)
output = peft_model.generate(
    input_ids=input["input_ids"],
    max_length=250,
    top_k=3,
    top_p=0.8,
    do_sample=True,
    repetition_penalty=1.3,
    num_return_sequences=1)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

In [ ]:
# 파일 제출은 해야되니까~
submit = pd.read_csv('sample_submission.csv')
# 제출 양식 파일(sample_submission.csv)을 활용하여 Embedding Vector로 변환한 결과를 삽입
submit.iloc[:,1:] = pred_embeddings
submit.to_csv('submit_0305.csv', index=False)